ELEVATE


In [ ]:
%pip install mediapipe opencv-python pandas scikit-learn

In [3]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [86]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic :

    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)
        # print(results.face_landmarks)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1))
         
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2))

        cv2.imshow('Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows() 

In [92]:
len(results.right_hand_landmarks.landmark)

21

In [88]:
len(results.left_hand_landmarks.landmark)

AttributeError: 'NoneType' object has no attribute 'landmark'

In [93]:
len(results.pose_landmarks.landmark)

33

In [199]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [205]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [206]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [207]:
class_name = "hand_behind_head"

In [208]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic :

    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)
        # print(results.face_landmarks)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  

        # # Pose Detections
        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
        #                           mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
        #                           mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1))
         
        # # Right hand
        # mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # # Left Hand
        # mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2))
        

        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            
            # # Extract Right Hand landmarks
            # right_hand = results.right_hand_landmarks.landmark if results.right_hand_landmarks else []
            # right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
            
            # # Extract Left Hand landmarks
            # left_hand = results.left_hand_landmarks.landmark if results.left_hand_landmarks else []
            # left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
            
            # Concatenate rows
            row = pose_row 
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except Exception as e:
            print(f"Error in landmark extraction: {e}")

        cv2.imshow('Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows() 

In [191]:
df = pd.read_csv('coords.csv')

In [192]:
df.head()



,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z73,v73,x74,y74,z74,v74,x75,y75,z75,v75
0,arms_crossed,0.459928,0.412244,-0.610899,0.999492,0.483150,0.361146,-0.562359,0.997719,0.497243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,arms_crossed,0.461247,0.412179,-0.630719,0.999467,0.484636,0.360832,-0.576182,0.997679,0.497837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,arms_crossed,0.461603,0.412194,-0.693050,0.999451,0.485290,0.360737,-0.637160,0.997695,0.498075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,arms_crossed,0.461733,0.412316,-0.644182,0.999447,0.485587,0.360743,-0.583881,0.997730,0.498175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,arms_crossed,0.461900,0.412324,-0.690207,0.999406,0.486019,0.360611,-0.633546,0.997648,0.498363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z73,v73,x74,y74,z74,v74,x75,y75,z75,v75
957,hand_behind_head,0.444324,0.270268,-1.055056,0.999988,0.477858,0.202139,-0.990792,0.999798,0.496787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
958,hand_behind_head,0.441421,0.275595,-0.992948,0.999986,0.476077,0.209709,-0.933569,0.999791,0.496078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
959,hand_behind_head,0.424092,0.277941,-0.665068,0.999981,0.458131,0.209111,-0.624565,0.999801,0.479814,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
960,hand_behind_head,0.416075,0.279199,-0.690043,0.999963,0.450653,0.208339,-0.654338,0.999797,0.471925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,hand_behind_head,0.418651,0.273141,-0.771125,0.999962,0.452756,0.200836,-0.725525,0.999809,0.473719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
df[df['class']=='Wave']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z73,v73,x74,y74,z74,v74,x75,y75,z75,v75


In [177]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [178]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [179]:
X_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z73,v73,x74,y74,z74,v74,x75,y75,z75,v75
445,0.455517,0.272344,-0.678369,0.999616,0.485190,0.212442,-0.638215,0.998454,0.500141,0.211495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382,0.439123,0.260381,-0.746265,0.999694,0.469788,0.197241,-0.695657,0.998742,0.485429,0.196698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,0.482142,0.108526,-1.029035,0.998267,0.514612,0.048263,-0.950965,0.994088,0.536124,0.052303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,0.459403,0.451066,-0.506288,0.999772,0.486545,0.403804,-0.447861,0.999256,0.498138,0.404846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,0.458801,0.511463,-0.721059,0.999949,0.486036,0.460688,-0.664213,0.999663,0.500554,0.460091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,0.407647,0.432286,-0.419463,0.999803,0.431932,0.387564,-0.370289,0.999389,0.447838,0.386557,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557,0.383612,0.513706,-0.723427,0.999961,0.414083,0.462201,-0.671621,0.999830,0.430431,0.460490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,0.461915,0.466107,-0.497428,0.999522,0.486643,0.407780,-0.437656,0.998624,0.498543,0.406245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,0.474817,0.539801,-0.676359,0.999963,0.500544,0.498990,-0.607703,0.999735,0.512911,0.500648,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:

from sklearn.impute import SimpleImputer


pipelines = {
    'lr': make_pipeline(SimpleImputer(strategy='constant', fill_value=0), StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(SimpleImputer(strategy='constant', fill_value=0), StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(SimpleImputer(strategy='constant', fill_value=0), StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(SimpleImputer(strategy='constant', fill_value=0), StandardScaler(), GradientBoostingClassifier()),
}

# Now your pipelines include an imputation step that fills NaN values with 0, a scaling step, and the model itself.


In [181]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [182]:
fit_models

{'lr': Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value=0, strategy='constant')),
                 ('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value=0, strategy='constant')),
                 ('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value=0, strategy='constant')),
                 ('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value=0, strategy='constant')),
                 ('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [183]:
fit_models['gb'].predict(X_test)

array(['arms_crossed', 'arms_crossed', 'arms_crossed', 'arms_crossed',
       'hand_behind_head', 'arms_crossed', 'hand_behind_head',
       'hand_behind_head', 'arms_crossed', 'hand_behind_head',
       'arms_crossed', 'hand_behind_head', 'arms_crossed',
       'hand_behind_head', 'hand_behind_head', 'arms_crossed',
       'hand_behind_head', 'arms_crossed', 'arms_crossed', 'arms_crossed',
       'hand_behind_head', 'hand_behind_head', 'arms_crossed',
       'arms_crossed', 'hand_behind_head', 'arms_crossed',
       'hand_behind_head', 'hand_behind_head', 'hand_behind_head',
       'arms_crossed', 'hand_behind_head', 'hand_behind_head',
       'hand_behind_head', 'hand_behind_head', 'arms_crossed',
       'arms_crossed', 'hand_behind_head', 'hand_behind_head',
       'arms_crossed', 'arms_crossed', 'arms_crossed', 'hand_behind_head',
       'arms_crossed', 'hand_behind_head', 'hand_behind_head',
       'arms_crossed', 'arms_crossed', 'hand_behind_head', 'arms_crossed',
       'hand_be

In [193]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [194]:
fit_models['gb'].predict(X_test)

array(['arms_crossed', 'arms_crossed', 'arms_crossed', 'arms_crossed',
       'hand_behind_head', 'arms_crossed', 'hand_behind_head',
       'hand_behind_head', 'arms_crossed', 'hand_behind_head',
       'arms_crossed', 'hand_behind_head', 'arms_crossed',
       'hand_behind_head', 'hand_behind_head', 'arms_crossed',
       'hand_behind_head', 'arms_crossed', 'arms_crossed', 'arms_crossed',
       'hand_behind_head', 'hand_behind_head', 'arms_crossed',
       'arms_crossed', 'hand_behind_head', 'arms_crossed',
       'hand_behind_head', 'hand_behind_head', 'hand_behind_head',
       'arms_crossed', 'hand_behind_head', 'hand_behind_head',
       'hand_behind_head', 'hand_behind_head', 'arms_crossed',
       'arms_crossed', 'hand_behind_head', 'hand_behind_head',
       'arms_crossed', 'arms_crossed', 'arms_crossed', 'hand_behind_head',
       'arms_crossed', 'hand_behind_head', 'hand_behind_head',
       'arms_crossed', 'arms_crossed', 'hand_behind_head', 'arms_crossed',
       'hand_be

In [195]:
y_test

445        arms_crossed
382        arms_crossed
499        arms_crossed
230        arms_crossed
639    hand_behind_head
             ...       
218        arms_crossed
557    hand_behind_head
170        arms_crossed
679    hand_behind_head
764    hand_behind_head
Name: class, Length: 289, dtype: object

In [196]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [197]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [189]:
model

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value=0, strategy='constant')),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [198]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Right Hand landmarks
            right_hand = results.right_hand_landmarks.landmark if results.right_hand_landmarks else []
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
            
            # Extract Left Hand landmarks
            left_hand = results.left_hand_landmarks.landmark if results.left_hand_landmarks else []
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
            
            # Concatenate rows
            row = pose_row + right_hand_row + left_hand_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer wa

hand_behind_head [0.03 0.97]
hand_behind_head [0.03 0.97]


c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


hand_behind_head [0.08 0.92]
hand_behind_head [0.15 0.85]


c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


...

c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


arms_crossed [0.52 0.48]
arms_crossed [0.5 0.5]


c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


arms_crossed [0.61 0.39]


c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
c:\Users\Hyder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
